In [1]:
from functools import partial

import astroplan as ap
import astropy.units as u
import numpy as np
import pandas as pd

from astropaul.database import database_connection
import astropaul.html as html
import astropaul.targetlistcreator as tlc
import astropaul.gemini as gemini

%load_ext autoreload
%autoreload 2


In [2]:
site_names = ["Gemini North", "Gemini South"]
sessions = {}
for site in site_names:
    session = tlc.ObservingSession(ap.Observer.at_site(site))
    for month in range(2, 8):
        session.add_full_day(f"2026-{month:02d}-01")
        session.add_full_day(f"2026-{month:02d}-15")
    # session.add_day_range("2026-01-01", "2026-01-05")
    # session.add_day_range("2026-01-01", "2026-06-30")
    sessions[site] = session

In [3]:
html_dir = "../../../../Observing Files/Gemini 2026A Proposal"
html.clear_directory(html_dir)

lower_mag_limit = 12
upper_mag_limit = 15

exposure_time_field = "gemini_iq70_speckle_program_time"

targets_pit_will_reject = {
    "TIC 292318612",
    "TIC 251757935",
    "TIC 285681367",
    "TIC 178953404",
    "TIC 139650665",
    "TIC 244279814",
}

common_steps = [
    tlc.add_targets,
    tlc.ancillary_data_from_tess,
    partial(tlc.add_database_table, table_name="zorroalopeke_observations"),
    partial(tlc.filter_targets, criteria=lambda df: df["Num Zorroalopeke Observations"] == 0),
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["Target Name"].isin(targets_pit_will_reject)),
    gemini.add_gemini_speckle_params,
]

list_steps = {
    "HQUS": [
        tlc.add_lists,
        partial(tlc.filter_targets, criteria=lambda df: df["List HQND 2025-09-30"]),
    ],
    "Priority 1 Faint": [
        tlc.add_lists,
        partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["List Resolved 2025-09-30"]),
        partial(tlc.filter_targets, criteria=lambda df: df["Vmag"] > lower_mag_limit),
        partial(tlc.filter_targets, criteria=lambda df: df["Target Source"].str.contains("Kostov 2022")),
        partial(tlc.add_database_table, table_name="dssi_observations"),
        # partial(tlc.filter_targets, criteria=lambda df: df["Num DSSI Observations"] == 0),
    ],
    "Priority 2 Faint": [
        partial(tlc.filter_targets, criteria=lambda df: df["Vmag"] > lower_mag_limit),
        partial(tlc.filter_targets, criteria=lambda df: df["Vmag"] < upper_mag_limit),
        partial(tlc.filter_targets, criteria=lambda df: df["Target Source"].str.contains("Kostov 2023")),
        partial(tlc.add_database_table, table_name="dssi_observations"),
        partial(tlc.filter_targets, criteria=lambda df: df["Num DSSI Observations"] == 0),
    ],
}

site_specific_steps = [
    partial(tlc.add_observability, observability_threshold=(35 * u.deg, 80 * u.deg), time_resolution=60 * u.min),
    partial(tlc.filter_targets, criteria=lambda df: df["Observable Any Night"]),
]

targets = set()
lists = {}
with database_connection() as conn:
    for list_prefix, steps in list_steps.items():
        for site_name, session in sessions.items():
            list_name = f"{list_prefix} {site_name}"
            creator = tlc.TargetListCreator(name=list_name, connection=conn)
            this_steps = (
                common_steps
                + steps
                + [partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["Target Name"].isin(targets))]
                + site_specific_steps
            )
            tl = creator.calculate(steps=this_steps, observing_session=session, verbose=False)
            tl.target_list["Group Category"] = list_prefix
            tl.target_list["Group Name"] = list_name
            tl.target_list["Observatory"] = pd.cut(
                tl.target_list["Dec"], bins=[-90, -25, 25, 90], labels=["South", "Both", "North"]
            )
            lists[list_name] = tl
            targets.update(tl.target_list["Target Name"])
            print(f"{len(tl.target_list):2d}   {tl.target_list[exposure_time_field].sum()/3600:4.1f}   {list_name}")


unified_list = tlc.TargetList.union(list(lists.values()), name="Gemini 2026A Proposal")

targets_to_skip = ["TIC 52856877", "TIC 50198590"]  # , "TIC 244279814"] # PIT disagreed with me that these are observable
unified_list.target_list = unified_list.target_list[~unified_list.target_list["Target Name"].isin(targets_to_skip)]
proposal_targets = unified_list.target_list

output_list = proposal_targets[["Target Name", "RA", "Dec", "Vmag", exposure_time_field]]
output_list.columns = ["Name", "RAJ2000", "DecJ2000", "V", "Exposure Time"]
output_list.to_csv("Gemini 2026A Targets.csv", index=False)
print(f"{len(proposal_targets)} targets, {proposal_targets[exposure_time_field].sum() / 3600:.2f} hours")

# print(unified_list.summarize())
html.render_observing_pages(unified_list, None, {}, html_dir)

13    2.9   HQUS Gemini North
12    2.9   HQUS Gemini South
14    3.8   Priority 1 Faint Gemini North
 2    0.6   Priority 1 Faint Gemini South
32    8.8   Priority 2 Faint Gemini North
11    3.0   Priority 2 Faint Gemini South
82 targets, 21.42 hours


In [4]:
import pandas as pd

with database_connection() as conn:
    ephem = pd.read_sql("""select t.target_name, ea.period Period_A, eb.period Period_B, eb.period / ea.period BA_Ratio
    from targets t
    join ephemerides ea on t.target_name = ea.target_name and ea.system = 'A' and ea.member = 'a'
    join ephemerides eb on t.target_name = eb.target_name and eb.system = 'B' and eb.member = 'a'
    where t.target_source like '%Kostov%' and t.target_type = 'QuadEB'
    order by eb.period / ea.period
    ;""", conn)

import plotly.express as px

display(px.scatter(ephem, x="Period_A", y="BA_Ratio"))
display(px.histogram( ephem, x="BA_Ratio", range_x=[0, 10], nbins=120))
display(px.histogram( ephem, x="BA_Ratio", range_x=[0, 10], nbins=240))

After going through the entire proposal process, the PIT software had problems with specific targets.  Remove them. 

* Start the PIT and open the file `Gemini 2026A Proposal BASE.xml`
  * This base file has investigators, title, and other fields already populated
* Import the target list created above into the PIT
* In targets tab, sort by Dec
* Select & Copy all targets Dec < -10
* Switch to Observations tab and Paste
* Specify weather conditions and Zorro speckle (South)
* Select & Copy all targets Dec > -10
* Switch to Observations tab and Paste
* Specify weather conditions and `Alopeke speckle (North)
* Do a Save As in the PIT, to the file `Gemini 2026A Proposal TARGETS.xml`
* The below code will add observing times for each target to the xml file

In [8]:
# edit the xml file saved by the PIT and add observation times
import xml.etree.ElementTree as ET

filename = "Gemini 2026A Proposal"

# first, get observation times for all targets (in hours, because that's all PIT accepts)
target_times = {
    str(name): (iq70 / 3600, iq85 / 3600)
    for _, (name, iq70, iq85) in proposal_targets[
        ["Target Name", "gemini_iq70_speckle_program_time", "gemini_iq85_speckle_program_time"]
    ].iterrows()
}

# for each weather condition:
# get the mapping of the nickname used in xml file to the TIC name
# then walk through each observation entry and add observing time to it for the appropriate conditions
tot = {"iq70": 0.0, "iq85": 0.0}
for index, name in enumerate(["iq70", "iq85"]):
    tree = ET.parse(f"{filename} TARGETS.xml")
    root = tree.getroot()
    target_map = {}
    for target in root.findall("targets/sidereal"):
        designator = target.attrib["id"]
        nickname = target.find("name").text
        target_map[designator] = nickname

    for observation in root.findall("observations/observation"):
        target_name = target_map[observation.attrib["target"]]
        target_time = target_times[target_name][index]
        tot[name] += target_time
        target_time_text = f"{target_time:.3f}"
        progTime = ET.Element("progTime")
        progTime.set("units", "s")
        progTime.text = target_time_text
        observation.append(progTime)
        time = ET.Element("time")
        time.set("units", "s")
        time.text = target_time_text
        observation.append(time)
    tree.write(f"{filename} {name}.xml")

tot

{'iq70': 21.423775626666682, 'iq85': 34.74545925333333}

* Use the above xml editing code to apply exposure times
* Make copy of the iq70 file with a band3 suffix
* Open the iq85 file and copy all the `<observation>` nodes
* Rename `Band 1/2` to `Band 3` for all nodes
* Paste these new band 3 nodes below the band 1/2 nodes in the new band3 file created above

In [6]:
observatories = ["North", "Both", "South"]
sub_lists = proposal_targets["Group Category"].unique()


def Count(x):
    return len(x)


def Hours(x):
    return np.sum(x) / 3600


proposal_targets["Group Category"] = [
    f"Faint ($V > {lower_mag_limit}$)" if "Faint" in name else name for name in proposal_targets["Group Category"]
]

grid = (
    proposal_targets.pivot_table(
        index="Group Category",
        columns="Observatory",
        values=exposure_time_field,
        aggfunc=[Count, Hours],
        fill_value=0,
        observed=False,
    )
    .swaplevel(0, 1, axis=1)
    .sort_index(axis=1, level=0)
)
grid.loc["Total"] = grid.sum()

formats = {
    ("South", "Count"): "{:.0f}",
    ("Both", "Count"): "{:.0f}",
    ("North", "Count"): "{:.0f}",
    ("South", "Hours"): "{:.2f}",
    ("Both", "Hours"): "{:.2f}",
    ("North", "Hours"): "{:.2f}",
}

foo = grid.style.format(formats)
print(foo.to_latex())
foo

\begin{tabular}{lrrrrrr}
Observatory & \multicolumn{2}{r}{South} & \multicolumn{2}{r}{Both} & \multicolumn{2}{r}{North} \\
 & Count & Hours & Count & Hours & Count & Hours \\
Group Category &  &  &  &  &  &  \\
Faint ($V > 12$) & 17 & 4.95 & 22 & 6.01 & 19 & 4.93 \\
HQUS & 13 & 3.08 & 1 & 0.20 & 10 & 2.25 \\
Total & 30 & 8.02 & 23 & 6.21 & 29 & 7.19 \\
\end{tabular}



* Paste the above table into the Overleaf for the experimental design
* For Phase 3, I just added the HQUS targets and their IQ85 exposure times, manually.
  * Ideally, my code could create separate Phase 1/2 and Phase 3 lists using different IQ70/85 exposure times, but I didn't have time for that this time.